In [23]:
import openai
import notion_client
import json

with open('openai_secret_key') as r:
    openai_secret_key = r.readline().strip()
    openai.api_key = openai_secret_key

with open('notion_secret_key') as r:
    notion_secret_key = r.readline().strip()
    notion = notion_client.Client(auth=notion_secret_key)

In [165]:
engine = 'text-davinci-003'
journal_id = '66a8cd52520543df9895c373627f821d'

In [47]:
entries = notion.databases.query(database_id=journal_id)

In [179]:
entry_id = 'c6affbed81284a09affcd41c8fa4915b'
entry = notion.pages.retrieve(page_id=entry_id)

In [180]:
blocks = notion.blocks.children.list(block_id=entry_id)['results']
plain_text = ''

for block in blocks:
    block_type = block['type']
    rich_text = block[block_type]['rich_text']
    for text in rich_text:
        plain_text += text['plain_text']
    plain_text += '  \n\n'

In [190]:
extraction_prompt = f"""
Extract the following entities in the following format:
Organizations: <List of all platforms, products, companies and institutions mentioned by name. Exclude broad terms such as industries ("Record Labels") that don't refer to specific companies.>
People: <List of all people mentioned by name>
Places: <List of all places mentioned by name>
Tags: <List of all topics the author is writing about>
Make the lists long and exhaustive.

Text:###
{plain_text}
###
"""

In [191]:
extraction = openai.Completion.create(
    engine=engine,
    prompt=extraction_prompt,
    max_tokens=1024,
    temperature=0.4
)
entities_text = extraction['choices'][0]['text']

In [192]:
print(entities_text)

Organizations: Songtradr, BMAT, Apple
People: Nic, Brian Schmitt, Carina, Minz, Julie, Mel, JJ, Zal
Places: Fort Greene, Prospect Park, Dominican Republic, United Kingdom, Ireland, Japan, South Korea, The Philippines, Scandinavia, South Korea
Tags: Cultures, Racial Stereotyping, Racial Minorities, Racism, Island Nations, Stereotypes, Beauty, Fandom, Escapism, Social Class, Parenting, Social Justice, Mood Swings


In [132]:
filtering_prompt = f"""
What follows is a series of lists:

{entities_text}

Remove any entries from the Organizations list so that it only contains specific companies and organizations. For example, if it contains "Universal Music" and "Record Labels", remove "Record Labels".
Filter the People list so that it only contains names of people (full names). For example, remove entries like "Friends" that refer to broad categories of people.
Remove from the Places list any entries that are not specific places.
Remove any entries that look like duplicates (e.g. New York, New York City)
Remove leading or trailing whitespace from all entries.
Capitalize all entries in title case.
If any of the topics present in the Tags list is present in any of the other lists, remove it from the Tags list.
Format the response as valid JSON.
"""

In [184]:
filtering = openai.Completion.create(
    engine=engine,
    prompt=filtering_prompt,
    max_tokens=1024,
    temperature=0.3
)
filtered_text = filtering['choices'][0]['text']

In [134]:
print(filtered_text)


[
  {
    "organizations": [
      "Twitter",
      "Airlines",
      "Substack",
      "New York City",
      "Adam Tooze",
      "The Atlantic"
    ],
    "people": [
      "Millenials",
      "Adam Tooze"
    ],
    "places": [
      "New York City",
      "Africa",
      "United States"
    ],
    "tags": [
      "social justice",
      "inequality",
      "religion",
      "community",
      "travel",
      "family",
      "moving"
    ]
  }
]
